In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import json
import random
from datetime import datetime, timedelta

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import os
import time
import re

def get_lottery_results(date_str):
    options = Options()
    options.add_argument('--headless=new')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--log-level=3')
    options.add_argument('--silent')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

    service = Service(log_output=os.devnull)
    driver = None

    try:
        driver = webdriver.Chrome(options=options, service=service)

        url = f"https://xoso.com.vn/xsmb-{date_str}.html"
        driver.get(url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()

        result_table = soup.find('table', class_='table-result')
        if not result_table:
            print(f"Không tìm thấy kết quả cho ngày {date_str}")
            return None

        # Lấy các mã code từ hàng đầu tiên
        codes = []
        first_row = result_table.find('tr')
        if first_row:
            print(first_row)
            code_cell = first_row.find('td', class_='number-prize', id='mb_prizeCode')
            if code_cell:
                code_spans = code_cell.find_all('span', class_=re.compile('code-.*'))
                codes = [span.get_text(strip=True) for span in code_spans]

        prize_structure = {
            'ĐB': {'count': 1, 'digits': 5},
            '1': {'count': 1, 'digits': 5},
            '2': {'count': 2, 'digits': 5},
            '3': {'count': 6, 'digits': 5},
            '4': {'count': 4, 'digits': 4},
            '5': {'count': 6, 'digits': 4},
            '6': {'count': 3, 'digits': 3},
            '7': {'count': 4, 'digits': 2}
        }

        results = {}
        # Bỏ qua hàng đầu tiên khi xử lý các giải
        for row in result_table.find_all('tr')[1:]:
            row_text = row.get_text(strip=True)
            if not row_text:
                continue

            if row_text.startswith('ĐB'):
                prize_name = 'ĐB'
                numbers_text = row_text[2:]
                numbers = [numbers_text]
                results[prize_name] = numbers
            elif any(row_text.startswith(str(i)) for i in range(1, 8)):
                prize_name = row_text[0]
                numbers_text = ''.join(filter(str.isdigit, row_text[1:]))

                prize_info = prize_structure.get(prize_name, {'count': 1, 'digits': 5})
                num_count = prize_info['count']
                digit_length = prize_info['digits']

                numbers = []
                current_pos = 0
                while current_pos < len(numbers_text) and len(numbers) < num_count:
                    if current_pos + digit_length <= len(numbers_text):
                        number = numbers_text[current_pos:current_pos+digit_length]
                        numbers.append(number)
                        current_pos += digit_length
                    else:
                        break

                while len(numbers) < num_count:
                    numbers.append('')

                results[prize_name] = numbers
            else:
                cells = row.find_all('td')
                if len(cells) >= 2:
                    prize_name = cells[0].get_text(strip=True)
                    numbers = cells[1].get_text(strip=True).split('-')
                    results[prize_name] = numbers

        special_prize = results.get('ĐB', [''])[0]

        return {
            'date': date_str,
            'special_prize': special_prize,
            'codes': codes,  # Thêm trường codes
            'all_results': results
        }
    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu cho ngày {date_str}: {e}")
        return None
    finally:
        if driver:
            driver.quit()

In [3]:
def get_lottery_results(date_str):
    options = Options()
    options.add_argument('--headless=new')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--log-level=3')
    options.add_argument('--silent')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

    service = Service(log_output=os.devnull)

    try:
        driver = webdriver.Chrome(options=options, service=service)

        url = f"https://xoso.com.vn/xsmb-{date_str}.html"
        driver.get(url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()

        result_table = soup.find('table', class_='table-result')
        if not result_table:
            print(f"Không tìm thấy kết quả cho ngày {date_str}")
            return None

        prize_structure = {
            'ĐB': {'count': 1, 'digits': 5},
            '1': {'count': 1, 'digits': 5},
            '2': {'count': 2, 'digits': 5},
            '3': {'count': 6, 'digits': 5},
            '4': {'count': 4, 'digits': 4},
            '5': {'count': 6, 'digits': 4},
            '6': {'count': 3, 'digits': 3},
            '7': {'count': 4, 'digits': 2}
        }

        results = {}
        for row in result_table.find_all('tr')[1:]:
            row_text = row.get_text(strip = True)
            if row_text.startswith('ĐB'):
                prize_name = 'ĐB'
                numbers_text = row_text[2:]
                numbers = [numbers_text]
                results[prize_name] = numbers
            elif any(row_text.startswith(str(i)) for i in range(1, 8)):
                prize_name = row_text[0]
                numbers_text = row_text[1:]

                prize_info = prize_structure.get(prize_name, {'count': 1, 'digits': 5})
                num_count = prize_info['count']
                digit_length = prize_info['digits']

                numbers = []
                current_pos = 0

                while current_pos < len(numbers_text) and len(numbers) < num_count:
                    if current_pos + digit_length <= len(numbers_text):
                        number = numbers_text[current_pos:current_pos+digit_length]
                        numbers.append(number)
                        current_pos += digit_length
                    else:
                        break

                while len(numbers) < num_count:
                    numbers.append('')

                results[prize_name] = numbers
            else:
                cells = row.find_all('td')
                if len(cells) >= 2:
                    prize_name = cells[0].get_text(strip=True)
                    numbers = cells[1].get_text(strip=True).split('-')
                    results[prize_name] = numbers

        special_prize = results.get('ĐB', [''])[0]

        return {
            'date': date_str,
            'special_prize': special_prize,
            'all_results': results
        }
    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu cho ngày {date_str}: {e}")
        return None
    finally:
        try:
            if 'driver' in locals() and driver:
                driver.quit()
        except:
            pass

In [4]:
def collect_data(output_file, end_date=None):
    all_results = []

    # Mặc định end_date là hôm nay - 1
    if end_date is None:
        end = datetime.today() - timedelta(days=1)
    else:
        end = datetime.strptime(end_date, '%d-%m-%Y')

    # Lấy ngày bắt đầu từ file json nếu có
    if os.path.exists(output_file):
        try:
            with open(output_file, 'r', encoding='utf-8') as f:
                all_results = json.load(f)

            if all_results:
                # Tìm ngày cuối cùng
                last_date = datetime.strptime(all_results[-1]['date'], '%d-%m-%Y')
                current = last_date + timedelta(days=1)
                print(f"⏩ Tiếp tục thu thập từ ngày {current.strftime('%d-%m-%Y')}...")
            else:
                print(f"⚠️ File rỗng. Vui lòng khởi tạo dữ liệu ban đầu.")
                return
        except Exception as e:
            print(f"❌ Lỗi khi đọc file {output_file}: {e}")
            return
    else:
        print(f"⚠️ File chưa tồn tại. Bạn cần chạy khởi tạo trước (hoặc truyền start_date).")
        return

    # Tiến hành thu thập
    count = 0
    while current <= end:
        date_str = current.strftime('%d-%m-%Y')
        print(f"📅 Đang thu thập dữ liệu cho ngày {date_str}...")

        result = get_lottery_results(date_str)
        if result:
            all_results.append(result)
            count += 1

        # Ghi định kỳ mỗi 7 ngày
        if count % 7 == 0:
            print("💾 Lưu dữ liệu định kỳ...")
            os.makedirs(os.path.dirname(output_file), exist_ok=True)
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(all_results, f, ensure_ascii=False, indent=2)

        # Đợi 3-5 giây ngẫu nhiên
        sleep_time = 3 + (2 * random.random())
        print(f"⏳ Đợi {sleep_time:.2f} giây...")
        time.sleep(sleep_time)
        current += timedelta(days=1)

    # Ghi dữ liệu cuối cùng
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)

    print(f"\n✅ Đã thu thập xong dữ liệu và lưu vào: {output_file}")

In [5]:
# today = datetime.now()
# end_date = (today - timedelta(days=1)).strftime('%d-%m-%Y')
# start_date = (today - timedelta(days=6205)).strftime('%d-%m-%Y')
output_file = r'D:\TechByte_LoDe\Lo-De-Prediction\xs_data\xsmb_data.json'
collect_data(output_file=output_file)

⏩ Tiếp tục thu thập từ ngày 07-07-2025...
📅 Đang thu thập dữ liệu cho ngày 07-07-2025...
⏳ Đợi 4.26 giây...

✅ Đã thu thập xong dữ liệu và lưu vào: D:\TechByte_LoDe\Lo-De-Prediction\xs_data\xsmb_data.json
